Partea a doua

In [3]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(subset='train')

In [4]:
newsgroups.data[:4]

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

Vizualizam cateva date.

In [5]:
newsgroups.target[:4]

array([7, 4, 4, 1])

In [6]:
list(newsgroups.target_names)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [7]:
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

categories =['alt.atheism','comp.graphics','comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


Sectionam datele pentru antrenarea datelor si testarea acestora.

In [8]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [9]:
vectorizer = TfidfVectorizer()
vectors_train = vectorizer.fit_transform(newsgroups_train.data)
vectors_test = vectorizer.transform(newsgroups_test.data)

In [10]:
clf = MultinomialNB()
clf.fit(vectors_train, newsgroups_train.target)
predicted = clf.predict(vectors_test)
expected = newsgroups_test.target

In [14]:
from sklearn import metrics

In [15]:
metrics.f1_score(expected, predicted, average='macro')

0.7557542971333199

In [11]:
def name_targets(target_names, targets):
    return [target_names[t] for t in targets]

In [12]:
df = pd.DataFrame({
    'expected': name_targets(newsgroups_test.target_names, expected),
    'predicted': name_targets(newsgroups_test.target_names, predicted),
    # shorten the texts to 1000 chars to reduce the volume of data to be sent to Facets Dive
    'data': [text[:1000] + '...' for text in newsgroups_test.data],
    # add text lengths
    'length': [len(text) for text in newsgroups_test.data],
}, columns=['expected', 'predicted', 'length', 'data'])

In [13]:
# sample of records to be visualized
df.head()

,expected,predicted,length,data
0,rec.autos,rec.autos,695,From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. ...
1,comp.windows.x,sci.crypt,939,From: Rick Miller <rick@ee.uwm.edu>\nSubject: ...
2,alt.atheism,alt.atheism,453,From: mathew <mathew@mantis.co.uk>\nSubject: R...
3,talk.politics.mideast,talk.politics.mideast,5239,From: bakken@cs.arizona.edu (Dave Bakken)\nSub...
4,talk.religion.misc,alt.atheism,1007,From: livesey@solntze.wpd.sgi.com (Jon Livesey...


In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
import requests
import pickle

import sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

def clean_line(t):
    return (t.replace('\n',' ')
            .replace('\r',' ')
            .replace('\t',' ')
            .replace('  ',' ')
            .strip())

In [17]:
import functools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os as os
import re 
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups

In [34]:
X_train=vectors_train
y_train=newsgroups_train.target
X_test=vectors_test
y_test=newsgroups_test.target

Pentru curarea datelor, eliminarea html urilor, caracterele sa fie scrise cu litere mici, etc.

In [19]:
#Define the preprocessing functions

html_detector = re.compile('<.*?>',flags=re.UNICODE)
punctuation_detector = re.compile('[\W_ ]',flags=re.UNICODE)  #keep spaces and alpanumeric characters

def drop_html(texts):
  return [re.sub(html_detector,' ',text) for text in texts]

def to_lowercase(texts):
  return [text.lower() for text in texts]

def drop_punctuation(texts):
  return [re.sub(punctuation_detector,' ',text) for text in texts]

def tokenize(texts):
  #here one can get smarter (e.g. wordpiece tokenizer)
  #we'll tokenize based on just the whitespace
  return [text.split(" ") for text in texts]

def drop_whitespaces(texts):
  #assume that the texts are already tokenized
  return [[word for word in text if not word.isspace() and len(word)>0 ] for text in texts]

stemmer = SnowballStemmer('english')
def stem(texts):
  #assume that the texts are already tokenized
  return [[stemmer.stem(word) for word in text] for text in texts]

def identity(x):
  return x

def compose(functions):
    return functools.reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

def get_preprocessing_pipeline():
  steps = [stem,drop_whitespaces,tokenize,to_lowercase,drop_punctuation,drop_html]
  return compose(steps)


[['this', 'text', 'need', 'to', 'be', 'clean', 'as', 'a', 'requir'],
 ['requir', 'pad']]

In [27]:
from sklearn.model_selection import train_test_split
import time

def train(classifier, X, y):
    start = time.time()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

    classifier.fit(X_train, y_train)
    end = time.time()

    print("Accuracy: " + str(classifier.score(X_test, y_test)) + ", Time duration: " + str(end - start))
    return classifier

In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

trial1 = Pipeline([ ('vectorizer', TfidfVectorizer()), ('classifier', MultinomialNB())])

train(trial1, newsgroups.data, newsgroups.target)

Accuracy: 0.8488731771984092, Time duration: 9.354552984237671


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(newsgroups.data, newsgroups.target, test_size=0.2, random_state=11)

In [39]:
preprocessing_pipeline = get_preprocessing_pipeline()
preprocessed_train = preprocessing_pipeline(X_train)
preprocessed_test = preprocessing_pipeline(X_test)

In [36]:
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)
predicted = nb_clf.predict(X_test)
np.mean(predicted == y_test)

0.7738980350504514

In [ ]:
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train, y_train)
predicted = sgd_clf.predict(X_test)
np.mean(predicted == y_test)

In [ ]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
predicted = rf_clf.predict(X_test)
np.mean(predicted == y_test)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
predicted = lr_clf.predict(X_test)
np.mean(predicted == y_test)

In [ ]:
kn_clf = KNeighborsClassifier()
kn_clf.fit(X_train, y_train)
predicted = kn_clf.predict(X_test)
np.mean(predicted == y_test)

Observam procentele de mai sus si alegem cea mai buna metoda.




#### Functia

In [ ]:
def functia(single_string):
    #indtroducem datele
    X_train= newsgroups.data
    y_train newsgroups.target
    #curatam textul
    preprocessing_pipeline = get_preprocessing_pipeline()
    preprocessed_train = preprocessing_pipeline(X_train)
    #aplicam mai multi algoritmi
    nb_clf = MultinomialNB()
    nb_clf.fit(X_train, y_train)

    sgd_clf = SGDClassifier()
    sgd_clf.fit(X_train, y_train)

    rf_clf = RandomForestClassifier()
    rf_clf.fit(X_train, y_train)

    lr_clf = LogisticRegression()
    lr_clf.fit(X_train, y_train)

    kn_clf = KNeighborsClassifier()
    kn_clf.fit(X_train, y_train)

    #alegem cel mai bun algoritmul potrivit orientandu-ne dupa procentele de mai sus